# Proyecto 1 Etapa 1

## 1. Lectura y Comprensión del Enunciado

El presente modelo aborda el reto de distribución que enfrenta LogistiCo, una empresa dedicada al transporte de insumos médicos y productos esenciales en la región de La Guajira, Colombia. Este territorio se caracteriza por su geografía extensa, condiciones climáticas adversas y la dispersión de comunidades en zonas remotas, lo cual dificulta la cobertura eficiente y oportuna de la demanda.

Garantizar el acceso equitativo a recursos críticos en estas comunidades es una necesidad urgente. En este contexto, LogistiCo cuenta con una flota mixta de camionetas y drones para realizar sus entregas. Esta situación se puede abstraer como un problema de optimización, en el que se busca minimizar el costo total del servicio logístico, garantizando la cobertura completa de la demanda, el respeto por ventanas de tiempo y las capacidades físicas de los vehículos.

## 2. Identificación de Elementos Clave

- **Componentes esenciales del sistema:**

Clientes o comunidades a atender.

Flota compuesta por camionetas y drones.

Capacidad, alcance y restricciones de cada tipo de vehículo.

Tiempos máximos y mínimos de entrega (ventanas de tiempo).

Costos asociados: fijos, mantenimiento, combustible, penalizaciones.

- **Limitaciones físicas:**

Los drones tienen un alcance limitado y una capacidad de carga menor.

Las camionetas, aunque más capaces, requieren rutas terrestres transitables.

Las distancias y tiempos de viaje deben respetar las ventanas de servicio del cliente.

- **Limitaciones organizacionales:**

Los recursos logísticos (vehículos) son limitados.

Se prioriza la minimización de costos, sin sacrificar la calidad del servicio.

Penalizaciones económicas si se incumplen ventanas de tiempo.

## 3. Formulación de la Estructura del Modelo

---

#### **Conjuntos**

- **𝐼**: Conjunto de clientes o comunidades.  
- **𝑉**: Conjunto de vehículos disponibles.

---

#### **Parámetros**

- **Distᵢᵥ**: Distancia del depósito al cliente *i* usando el vehículo *v*.  
- **𝐹ₜ**: Tarifa fija de flete por kilómetro.  
- **𝐶ₘ**: Costo de mantenimiento por kilómetro.  
- **𝑃𝑓**: Precio del combustible por litro.  
- **𝑅ᵥ**: Rendimiento de combustible del vehículo *v*.  
- **𝑇ₘᵢₙ,ᵢ**, **𝑇ₘₐₓ,ᵢ**: Ventana de tiempo de atención del cliente *i*.  
- **Demandaᵢ**: Cantidad de insumos que requiere el cliente *i*.  
- **Capᵥ**: Capacidad de carga del vehículo *v*.  
- **Rangoᵥ**: Rango máximo que puede recorrer el vehículo *v*.

---

#### **Variables de Decisión**

- **𝑥ᵢᵥ ∈ {0,1}**: Indica si el vehículo *v* atiende al cliente *i*.  


---

#### **Variables Auxiliares**

- **𝛿ᵥ**: Toma valor 1 si el vehículo *v* es camioneta, y 0 si es dron.

---


### Formulación de Restricciones:

#### **Ventanas de Tiempo:**

$T_{min,i} \leq t_{iv} \leq T_{max,i} \quad \forall i \in I,\; \forall v \in V$

#### **Capacidad de los Vehículos:**

$\sum_{i \in I} Demanda_i \cdot x_{iv} \leq Cap_v \quad \forall v \in V$

#### **Autonomía de los Vehículos:**

$\sum_{i \in I} Dist_{iv} \cdot x_{iv} \leq Rango_v \quad \forall v \in V$

#### **Asignación:** (DUDAA)

$\sum_{v \in V} x_{iv} = 1 \quad \forall i \in I$

En términos de que un cliente solo puede ser atendido por un vehículo para una misión específica

DUDA: Deberia haber una restricción tipo la suma de la variable de decisión debe ser mayor a 1 para un día, o sea en un día deben llevarle a 3 clientes...minimo que se usen 2 vehiculos

#### **Restricción por condiciones climáticas, de geografía, etc.**
Matriz binaria $Accesibilidad_{iv}$:

- $Accesibilidad_{iv} = 1$ si el vehículo $v$ puede acceder al cliente $i$  
- $Accesibilidad_{iv} = 0$ si no puede (por clima, geografía, etc.)

$$
x_{iv} \leq Accesibilidad_{iv} \quad \forall i \in I,\ \forall v \in V
$$

**Ejemplos:**

- Si hay una montaña entre el cliente y el depósito, $Accesibilidad_{iv} = 0$ para **camionetas**.
- Si el clima impide volar, $Accesibilidad_{iv} = 0$ para **drones** ese día.

#### **Retorno para reabastecer (si no cabe toda la carga)**

Si la demanda de múltiples clientes excede la capacidad de un vehículo, se permite que:

- **El mismo vehículo realice varios viajes.**
- **Pero un cliente solo puede ser atendido una vez** (por un solo viaje de algún vehículo).

Para evitar que el mismo cliente sea visitado más de una vez:

$$
x_{iv} =
\begin{cases}
1 & \text{si el vehículo } v \text{ atiende al cliente } i \\
0 & \text{en otro caso}
\end{cases}
$$

Y se impone:

$$
\sum_{v \in V} x_{iv} = 1 \quad \forall i \in I
$$

Lo cual garantiza que **cada cliente sea atendido por un solo vehículo en un solo viaje**.

## 4. Preprocesamiento de datos

In [ ]:
origen = {
    0: (-72.9071, 11.5449)
}

destino = {
    1: (-72.8500, 11.6000),
    2: (-72.9200, 11.5200),
    3: (-72.8800, 11.5800),
}

In [35]:
import requests

# API key
api_key = '5b3ce3597851110001cf6248be808c25e2964fb2b3e74f3c74d8ce63'

headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}

# Nodos
nodes = {
    1: (-72.9071, 11.5449), # Origen
    2: (-72.8500, 11.6000), # Destino 1
    3: (-72.9200, 11.5200), # Destino 2
    4: (-72.8800, 11.5800) # Destino 3
}

url = 'https://api.openrouteservice.org/v2/directions/driving-car'

# Diccionario para almacenar resultados
dtcamioneta = {}

for i, coords_i in nodes.items():
    for j, coords_j in nodes.items():
        # Si son el mismo nodo
        if i == j:
            dtcamioneta[(i, j)] = (999, 999)
            continue
            
        start = f"{coords_i[0]},{coords_i[1]}"
        end = f"{coords_j[0]},{coords_j[1]}"

        params = {
            'start': start,
            'end': end
        }

        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            segmento = data['features'][0]['properties']['segments'][0]
            distance = round(segmento['distance'] / 1000, 2)
            duration = round(segmento['duration'] / 60, 2)
            dtcamioneta[(i, j)] = (distance, duration)
        else:
            try:
                error_message = response.json().get("error", {}).get("message", "")
            except Exception:
                error_message = ""
            if "Could not find routable point" in error_message:
                dtcamioneta[(i, j)] = (999, 999)
            else:
                dtcamioneta[(i, j)] = (999, 999)



In [36]:
import math

# Función para calcular la distancia entre dos puntos geográficos usando longitud y latitud
def haversine(coord1, coord2):

    lon1, lat1 = coord1
    lon2, lat2 = coord2
    
    R = 6371.0
    
    # Grados a radianes
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    # Fórmula matemática
    a = math.sin(delta_phi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c
    return distance


nodes = {
    1: (-72.9071, 11.5449),
    2: (-72.8500, 11.6000),
    3: (-72.9200, 11.5200),
    4: (-72.8800, 11.5800)
}

drone_speed_kmph = 105
drone_speed_km_min = drone_speed_kmph / 60.0  # km por minuto

# Diccionario para almacenar los resultados
dtdron = {}

for i, coord_i in nodes.items():
    for j, coord_j in nodes.items():
        # Si son el mismo nodo
        if i == j:
            dtdron[(i, j)] = (999, 999)
        else:
            distance_km = haversine(coord_i, coord_j)
            duration_min = distance_km / drone_speed_km_min  # Tiempo en minutos
            dtdron[(i, j)] = (round(distance_km, 3), round(duration_min, 3))
            


In [37]:
print("Distancia Tiempo Camionetas: ", dtcamioneta)
print("Distancias tiempo para drones:", dtdron)

Distancia Tiempo Camionetas:  {(1, 1): (999, 999), (1, 2): (21.08, 27.03), (1, 3): (4.3, 7.89), (1, 4): (999, 999), (2, 1): (21.17, 27.48), (2, 2): (999, 999), (2, 3): (22.98, 34.06), (2, 4): (999, 999), (3, 1): (4.49, 8.6), (3, 2): (22.98, 34.06), (3, 3): (999, 999), (3, 4): (999, 999), (4, 1): (999, 999), (4, 2): (999, 999), (4, 3): (999, 999), (4, 4): (999, 999)}
Distancias tiempo para drones: {(1, 1): (999, 999), (1, 2): (8.731, 4.989), (1, 3): (3.105, 1.774), (1, 4): (4.894, 2.796), (2, 1): (8.731, 4.989), (2, 2): (999, 999), (2, 3): (11.717, 6.695), (2, 4): (3.953, 2.259), (3, 1): (3.105, 1.774), (3, 2): (11.717, 6.695), (3, 3): (999, 999), (3, 4): (7.969, 4.554), (4, 1): (4.894, 2.796), (4, 2): (3.953, 2.259), (4, 3): (7.969, 4.554), (4, 4): (999, 999)}


## 5. Desarrollo de la Función Objetivo

### Función Objetivo Propuesta

#### ¿Qué buscamos minimizar?

El **costo total**, que estará compuesto por:

1. **Costo por distancia**:
$$
\text{Distancia} \times (\text{Tarifa de flete} + \text{Costo de mantenimiento})
$$

2. **Costo por combustible** (solo para camionetas):
$$
\frac{\text{Distancia}}{\text{Rendimiento}} \times \text{Precio del combustible}
$$

3. **Penalización por tiempo de entrega excedido**:

Se aplica una penalización económica cuando un pedido es entregado fuera de su ventana de tiempo, específicamente después del tiempo máximo permitido. Esta penalización se define como:

$$
\left( \frac{\text{Retraso (min)}}{10} \right) \times 10{,}000
$$

donde el retraso en minutos se divide entre 10 y se multiplica por un valor penalizador de $10{,}000$ unidades monetarias, incentivando así la puntualidad en las entregas.


---

### Formulación Simplificada de la Función Objetivo


$$
\text{Minimizar:} \sum_{i \in I} \sum_{v \in V} x_{iv} \cdot \left( \text{Dist}_{iv} \cdot (Ft + Cm) + \delta_v \cdot \left( \frac{\text{Dist}_{iv}}{R_v} \cdot Pf \right) + \text{Penalización}_{iv} \right)
$$

donde:

- $x_{iv}$: variable binaria que vale 1 si el vehículo $v$ atiende al cliente $i$, y 0 en caso contrario.  
- $\text{Dist}_{iv}$: distancia entre el depósito y el cliente $i$ cuando es atendido por el vehículo $v$.  
- $Ft$: tarifa de operación fija por kilómetro ($5{,}000$ unidades monetarias).  
- $Cm$: costo de mantenimiento por kilómetro ($700$ unidades monetarias).  
- $Pf$: precio del combustible ($15{,}000$ unidades monetarias por litro).  
- $R_v$: rendimiento del vehículo $v$ (en km/l).  
- $\delta_v$: parámetro que vale 1 si el vehículo $v$ es una camioneta, y 0 si es un dron (ya que los drones no consumen combustible). 

- $\text{Penalización}_{iv}$: penalización por retraso, definida como:

  $$
  \text{Penalización}_{iv} = \left( \frac{\max(0,\; t_{iv} - T_{\text{máx},i})}{10} \right) \cdot 10{,}000
  $$

  donde $t_{iv}$ es el tiempo estimado de llegada del vehículo $v$ al cliente $i$, y $T_{\text{máx},i}$ es el tiempo máximo permitido para atender al cliente $i$.


---



### Ejemplo de cálculo del costo para un cliente con camioneta

**Datos de entrada**:
- Distancia del depósito a Caserío El Sol = \(10 \, \text{km}\)
- Vehículo asignado: camioneta (200 unidades de capacidad, 300 km de rango)
- Envío se realiza 15 minutos después del máximo permitido
- Rendimiento de la camioneta: \(5 \, \text{km/L}\)

**Costo fijo por distancia**:
$$
10 \cdot (5{,}000 + 700) = 10 \cdot 5{,}700 = 57{,}000
$$

**Costo de combustible**:
$$
\left( \frac{10}{5} \right) \cdot 15{,}000 = 2 \cdot 15{,}000 = 30{,}000
$$

**Penalización por retraso**:
$$
\left( \frac{15}{10} \right) \cdot 10{,}000 = 1.5 \cdot 10{,}000 = 15{,}000
$$

**Costo total**:
$$
57{,}000 + 30{,}000 + 15{,}000 = 102{,}000
$$



---

## 6. Validación y Análisis

## 7.Ejemplos ilustrativos